This notebook is about **book recommender system** using collaborative filtering 

Notebook Author:Tharun K

Dept:Electronics and Communication Engineering

College:KPR INSTITUTE OF ENGINEERING AND TECHNOLOGY

In [14]:
import numpy as np
import pandas as pd

# Load the datasets 

In [15]:
books_df = pd.read_csv('Books.csv')
books_df.head()

C:\Users\tharun k\AppData\Local\Temp\ipykernel_7716\3005238537.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv('Books.csv')


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [20]:
ratings_df = pd.read_csv('Ratings.csv')
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


# Data Preprocessing

In [17]:
books_df.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [21]:
books_df.dropna(inplace=True)

In [22]:
books_df.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
Image-URL-S            0
Image-URL-M            0
Image-URL-L            0
dtype: int64

In [23]:
ratings_df.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

Let's go on the assumption that if person $A$ likes movies $M_0, M_1, ..., M_i$,
then there goes person $B$ who likes one or more movies from $M_i$ let's call them $M_j$.

This would mean that $A$ and $B$ has a movie that they both liked, therefore other movies from both $M_j$ and $M_i$ can be liked by both $A$ and $B$ with high probability.

------------------

In [24]:
df = books_df.merge(ratings_df, on='ISBN')

In [29]:
df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,0
...,...,...,...,...,...,...,...,...,...,...
1031124,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,276463,7
1031125,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,276579,4
1031126,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,276680,0
1031127,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,276680,0


# Collaborative Filtering

In [39]:
B_j = "There's a Bat in Bunk Five" # book-Title as input, now it's just one book
recommended_books = []

# Find the book in the database, and sort it by book-rating
book_db = df[df['Book-Title'] == B_j]\
            .sort_values(by='Book-Rating', ascending=False)

# Get the first 5 users who liked this book
for user in book_db.iloc[:5]['User-ID'].values:
    
    # Get the rated books for this user
    rated_books = df[df['User-ID'] == user]
    
    # Get the five biggest rated books by this user
    rated_books = rated_books[rated_books['Book-Title'] != B_j].sort_values(by='Book-Rating', ascending=False).iloc[:5]
    
    # Add these to the recommendations
    recommended_books.extend(list(rated_books['Book-Title'].values))
    
recommended_books = np.unique(recommended_books)
    
for book in recommended_books:
    print(book)

84 Charing Cross Road
A Prayer for Owen Meany
As Nature Made Him : The Boy Who Was Raised as a Girl
Behind the Bedroom Wall
Billy
Bridget Jones's Diary
Confessions of a Shopaholic (Summer Display Opportunity)
Friday Night Lights: A Town, a Team, and a Dream
Girl Goddess #9 : Nine Stories
House Across the Cove
I Who Never Known Men
Man With A Message (The Men Of Maple Hill) (Harlequin Superromance, No. 1056)
Maniac Magee
Murder in the Air (Sophie Greenway Mystery)
Mysteries of Animal Intelligence
Playing Beatie Bow (Puffin Books)
River, Cross My Heart
Sabriel (adult edition) (The Abhorsen Trilogy)
Sex &amp; the City
Story of My Life
The Dark Half
The Pilot's Wife : A Novel
The Two Dead Girls (Green Mile Series)
The Witch of Blackbird Pond (Yearling Newbery)
Victory Over Japan : A Book of Stories


Now weight each book by the similiarity on the publisher feature

In [40]:
pbook_publisher = df[df['Book-Title'] == B_j].iloc[0]['Publisher'].split('|')
scores = {}  # {title: score ...}

for book in recommended_books:
    booked = df[df['Book-Title'] == book].iloc[0]
    book_publisher = booked['Publisher'].split('|')
    score = 0
    
    # How many pbook_publisher can be found in book_publisher?
    for pbook_publisher in pbook_publisher:
        if pbook_publisher in book_publisher:
            score += 1
    
    scores[book] = score
    
# Sort them on score and reverse it, because the bigger the score the better 
recommended_books = sorted(scores, key=lambda x: scores[x])[::-1]  

The recommendations are now weighted

In [41]:
for book in recommended_books:
    print(book)

Victory Over Japan : A Book of Stories
The Witch of Blackbird Pond (Yearling Newbery)
The Two Dead Girls (Green Mile Series)
The Pilot's Wife : A Novel
The Dark Half
Story of My Life
Sex &amp; the City
Sabriel (adult edition) (The Abhorsen Trilogy)
River, Cross My Heart
Playing Beatie Bow (Puffin Books)
Mysteries of Animal Intelligence
Murder in the Air (Sophie Greenway Mystery)
Maniac Magee
Man With A Message (The Men Of Maple Hill) (Harlequin Superromance, No. 1056)
I Who Never Known Men
House Across the Cove
Girl Goddess #9 : Nine Stories
Friday Night Lights: A Town, a Team, and a Dream
Confessions of a Shopaholic (Summer Display Opportunity)
Bridget Jones's Diary
Billy
Behind the Bedroom Wall
As Nature Made Him : The Boy Who Was Raised as a Girl
A Prayer for Owen Meany
84 Charing Cross Road
